# Create Customer Segmets

## Problem Statement
> #### Build a model to group the customers into groups based on their purchase history

#### The solution notebook should contain the below sections

1. Exploratory Data Analysis

2. Data Preprocessing and Transformation

3. Model Building

4. Model Evaluation Metrics

5. Hyper Parameter Tuning

**Note**: Feel free to add new sections

## Data Set Information:

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

### Attribute Information:

* InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
* StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
* Description: Product (item) name. Nominal.
* Quantity: The quantities of each product (item) per transaction. Numeric.
* InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
* UnitPrice: Unit price. Numeric, Product price per unit in sterling.
* CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
* Country: Country name. Nominal, the name of the country where each customer resides.

<img src='dataset.PNG'>

### I. Exploratory Data Analysis

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
try:
    data = pd.read_csv("data.csv")
    data.drop(['StockCode', 'Description'], axis = 1, inplace = True)
    print "Wholesale customers dataset has {} samples with {} features each.".format(*data.shape)
except:
    print "Dataset could not be loaded. Is the dataset missing?"
    
data.head()

Wholesale customers dataset has 541909 samples with 6 features each.


,InvoiceNo,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### II. Data Preproessing and Transformation

In [2]:
#Setting now
NOW = dt.datetime(2011,12,10)
#Droping the duplicate datas
customer_country=data[['Country','CustomerID']].drop_duplicates()
#Grouping based on Contry
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


In [3]:
# Huge number of the Customer from so Taking United Kingdom customer alone
data = data.loc[data['Country'] == 'United Kingdom']
#removing missing customer values since we are using grouping based customer
data = data[pd.notnull(data['CustomerID'])]
#removing the negative unit price since negative are returned from customers
data = data[(data['Quantity']>0)]

In [4]:
#Calculating the total price 
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']

### III. Model Building

In [5]:
NOW = dt.datetime(2011,12,10)
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [6]:
rfmTable = data.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})
rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency','InvoiceNo': 'frequency','TotalPrice': 'monetary_value'}, inplace=True)

In [7]:
rfmTable.head()

,frequency,monetary_value,recency
CustomerID,,,
12346.0,1,77183.60,325
12747.0,103,4196.01,2
12748.0,4596,33719.73,0
12749.0,199,4090.88,3
12820.0,59,942.34,3


The first customer has shopped only once, bought one product at a huge quantity(74,215). The unit price is very low.
So we are going to split the metrics based on quartlies

### IV. Model Evaluation Metrics

In [8]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()
segmented_rfm = rfmTable

In [9]:
#Calculating the RScore
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

#Calculating the FMScore
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,frequency,monetary_value,recency,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12346.0,1,77183.60,325,4,4,1
12747.0,103,4196.01,2,1,1,1
12748.0,4596,33719.73,0,1,1,1
12749.0,199,4090.88,3,1,1,1
12820.0,59,942.34,3,1,2,2


Add the new colum to combine the RFM score. 

### V.Hyper Parameter Tuning

In [10]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,frequency,monetary_value,recency,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346.0,1,77183.60,325,4,4,1,441
12747.0,103,4196.01,2,1,1,1,111
12748.0,4596,33719.73,0,1,1,1,111
12749.0,199,4090.88,3,1,1,1,111
12820.0,59,942.34,3,1,2,2,122


In [11]:
#selecting the 111 score customer since RFM consider 111 is best Customer.
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,frequency,monetary_value,recency,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,431,259657.30,0,1,1,1,111
17450.0,337,194550.79,8,1,1,1,111
17511.0,963,91062.38,2,1,1,1,111
16684.0,277,66653.56,4,1,1,1,111
14096.0,5111,65164.79,4,1,1,1,111
13694.0,568,65039.62,3,1,1,1,111
15311.0,2379,60767.90,0,1,1,1,111
13089.0,1818,58825.83,2,1,1,1,111
15769.0,130,56252.72,7,1,1,1,111


Above are the top 10 best customer as per the RFM Score